# Excercise 3 Image Filtering

In [ ]:
import scipy.ndimage as ndi
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io

### Smoothing filters

In [ ]:
# Read image from disk and convert grey levels to double
x = np.float64(io.imread("images/lena.jpg"))

# Create the filter kernel
k = 3
h = np.ones((k, k)) / (k ** 2) # mean filter kernel

# Compute the filter output
y = ndi.correlate(x, h, mode="reflect")
plt.figure(1)
plt.subplot(1, 2, 1)
plt.title("Original image")
plt.imshow(x, clim=[0, 255], cmap="gray")
plt.subplot(1, 2, 2)
plt.title(f"Mean filtered image k = {k}")
plt.imshow(y, clim=[0, 255], cmap="gray")

### Smoothing and tresholding
An important application of spatial averaging filters is to blur the image so that small, uninteresting objects are confused with the background and to emphasize larger objects, which can then be easily detected.

In [ ]:
x = np.float64(io.imread("images/space.jpg"))

# Compute a mean filter with a kernel size of 10x10
k = 10
h = np.ones((k, k)) / (k ** 2)
y = ndi.correlate(x, h, mode="reflect")

# Compute the maximum grey level
max_grey = np.max(x)

# Compute a mask that eliminates all grey levels below 25% of max value
mask = y > (max_grey * (25.0 / 100.0))

# Compute the tresholding
z = mask * y

# Show results
plt.figure(1)
plt.subplot(1, 3, 1)
plt.title("Original image")
plt.imshow(x, clim=[0, 255], cmap="gray")
plt.subplot(1, 3, 2)
plt.title(f"Mean filtered image")
plt.imshow(y, clim=[0, 255], cmap="gray")
plt.subplot(1, 3, 3)
plt.title(f"Tresholded image at 25%")
plt.imshow(z, clim=[0, 255], cmap="gray")
plt.subplots_adjust(wspace=0.8, hspace=0.8)